In [1]:
!pip install gensim

In [3]:
import gensim
import pandas as pd

In [7]:
df=pd.read_json("Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [8]:
#only_reviewText_is_useful_for_word_vector_remaining_we_can_discard
df.shape

(194439, 9)

In [9]:
df.reviewText#interested_column

0         They look good and stick good! I just don't li...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome! stays on, and looks great. can be use...
                                ...                        
194434    Works great just like my original one. I reall...
194435    Great product. Great packaging. High quality a...
194436    This is a great cable, just as good as the mor...
194437    I really like it becasue it works well with my...
194438    product as described, I have wasted a lot of m...
Name: reviewText, Length: 194439, dtype: object

In [10]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [11]:
#preprocessing_of_this_text
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")
#tokenising_text

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [13]:
review_text=df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [15]:
model=gensim.models.Word2Vec(
    window=10, #10_words_before_our_target_word_and_10_words_after_our_target_word, window is a parameter
    min_count=2, #atleast_2_words_must_be_there_in_a_sentence_in_order_to_be_considered_for_training
    workers=4 #no.of_CPU_threads_used
)

In [16]:
#makin_the_vocabulary
model.build_vocab(review_text, progress_per=1000)

In [17]:
model.epochs #by_default_epochs_are_set_to_5

5

In [18]:
model.corpus_count #how_many_total_examples_does_we_have

194439

In [19]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61503755, 83868975)

In [22]:
model.save("./tensorboard-amazon-cell-accessories-reviews-short.model")

In [23]:
model.wv.most_similar("bad")

[('terrible', 0.6665558815002441),
 ('shabby', 0.6375533938407898),
 ('horrible', 0.6022359728813171),
 ('good', 0.5795396566390991),
 ('awful', 0.5387818813323975),
 ('legit', 0.5281513929367065),
 ('okay', 0.5247970223426819),
 ('sad', 0.5177406668663025),
 ('poor', 0.5111410617828369),
 ('crappy', 0.5089898705482483)]

In [24]:
model.wv.similarity(w1='cheap', w2='inexpensive') #cosine_similarity_score

0.5272028

In [25]:
model.wv.similarity(w1='great', w2='good')

0.7787986